In [1]:
import os
import pandas as pd
test_data_path = r'C:\Users\W Craig\Desktop\combined_files'

#This block of code creates a list of paths to cdrx files which do not have an accompanying .CSV file. 
#The objective of this module is to find all of these 'unaccompanied' files and then create a .CSV version suitable for import and analysis. 

#Whenever the database is re-scanned for new files, the unaccompanied_cdrx_path_list must be cleared. Otherwise, it will simply add the new list of unaccompanied files to an old list.

unaccompanied_cdrx_path_list = []
complete_cdrx_path_list = []

for root, dirs, files in os.walk(test_data_path):
        for filename in files:
                if filename.lower().endswith('cdrx'):
                        complete_cdrx_path_list.append(r"{0}\{1}".format(root, filename))
                        csv_filename = os.path.splitext(filename)[0] + '.CSV'
                        if csv_filename not in files:     
                        # os.path.join is performed to handle the case of one vehicle having multiple records. 
                        # The record is saved as the vehicle's VIN, but successive records bearing the same name are placed in different directories.  
                        # Considering the filepath as part of the record's 'identity' will ensure that these edge cases are accommodated.

                                # unaccompanied_cdrx_path_list.append(os.path.join(root, filename))
                                #Updating this to raw string for 'cleanliness' and resilience. This prevents any need to escape the backslashes.
                                unaccompanied_cdrx_path_list.append(r"{0}\{1}".format(root, filename))



dfs = []

for file_path in complete_cdrx_path_list:

        df = pd.DataFrame({'filepath': [file_path]})

        # Append the DataFrame to the list of DataFrames
        dfs.append(df)
# Combine the list of DataFrames into a single DataFrame
complete_path_list_dataframe = pd.concat(dfs, ignore_index=True)

In [4]:
from m0_1_system_variables import bosch_filepath
import psutil
import sys
import subprocess
import time
import pyautogui as gui
import pyperclip as pyp
import os



'''This block is used to determine whether the Bosch CDR tool is running'''
def check_process(process_name):
    """Check if a process is running on Windows"""
    for proc in psutil.process_iter(['name']):
        if proc.name() == process_name:
            return True
    return False
    sys.exit(1)

    # Check if 'cdr.exe' is running
if check_process('CDR.EXE'):
    print('CDR.EXE is running. Killing CDR.EXE and re-starting process to make it visible')
    subprocess.run(['taskkill', '/f', '/im', 'cdr.exe'])
else:
    process = subprocess.Popen(bosch_filepath)
    time.sleep(2)

#Must truncate the filepath for use in CDR to eliminate the filename and last backslash.
for working_file in unaccompanied_cdrx_path_list[1]:
    working_file_dir_str = str(os.path.dirname(working_file))
    gui.hotkey('ctrl', 'o')
    gui.keyDown('shift')
    for i in range(7):    # Press the Tab key six times
        gui.press('tab')
        gui.keyUp('shift')       # Release the Shift key
    gui.press('Enter')
    pyp.copy(working_file_dir_str)
    gui.hotkey('ctrl', 'v')
    gui.press('Enter')
    time.sleep(1)
    # for i in range(4):
    #     gui.press('tab')
    #     time.sleep(0.1)
    # time.sleep(1)
    # gui.press('space')
    # gui.press('enter')
    # time.sleep(1)
    # gui.press('alt')
    # gui.press('right')
    # for i in range(9):
    #     gui.press('down')
    #     time.sleep(0.1)
    # for i in range(2):
    #     gui.press('enter')
    #     time.sleep(0.1)


KeyboardInterrupt: 